# CEU - DV3 Final Project - UFO Sightings
### Author: Son Nam Nguyen
### Date: 2022-05-29
### Target: Multiple Charts + Dash app

## Setup

Install all the necessary libraries needed for the notebook.

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np


# import the Dash core and HTML components
from dash import dcc
from dash import html

# we need to import the input and output functions for the interactive features
from dash import Input, Output


# import the JupyterDash extension
from jupyter_dash import JupyterDash


## Introduction and Questions

Table containing 80,000+ records of UFO sightings between 1949 and 2014, including city, state, country, lat/long, shape, duration, date/time, and comments

**Business questions are inspired by Maven Analytics.**

1. Figure out which shapes of UFOs tend to be seen most often.

2. Take a look at data since the year 1990. How have annual sightings trended?

3. Which US states report the most UFO sightings?

4. Understand the distribution of the duration of UFO sightings.

## Data Ingestion

I have uploaded the data from [Maven Analytics](https://www.mavenanalytics.io/data-playground?accessType=open&dataStructure=5wfxyeVf1etbP4TXdyPdG1&page=3) to my public Github repo.

In [24]:
#read data from Github
df_ufo = pd.read_csv('https://github.com/nszoni/ceu-dv3/raw/main/data/ufo_sightings_scrubbed.csv', low_memory=False)
df_ufo.head(5)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611


Column names are a bit dirty (e.g. spaces, whitespaces), not something one would expect in a real data project.

In [26]:
# show column names
df_ufo.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

Let's give a try to clean up the column names.

In [27]:
#pass a list to column names
df_ufo.columns = ['date', 'city', 'state', 'country', 'shape', 'duration_sec', 'duration_min_hour', 'comments', 'date_posted', 'lat', 'long']

Column names are now OK, but we need to convert columns to their right format (currently all have a object -- string in numpy dtype).

In [28]:
df_ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               80332 non-null  object 
 1   city               80332 non-null  object 
 2   state              74535 non-null  object 
 3   country            70662 non-null  object 
 4   shape              78400 non-null  object 
 5   duration_sec       80332 non-null  object 
 6   duration_min_hour  80332 non-null  object 
 7   comments           80317 non-null  object 
 8   date_posted        80332 non-null  object 
 9   lat                80332 non-null  object 
 10  long               80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


In [29]:
#convert multiple datatypes
df_ufo['duration_sec'] = pd.to_numeric(df_ufo['duration_sec'], errors='coerce')
df_ufo['date_posted'] = pd.to_datetime(df_ufo['date_posted'])
df_ufo['date'] = pd.to_datetime(df_ufo['date'])
df_ufo['lat'] = pd.to_numeric(df_ufo['lat'], errors='coerce')
df_ufo['long'] = pd.to_numeric(df_ufo['long'], errors='coerce')

In [30]:
df_ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               80332 non-null  datetime64[ns]
 1   city               80332 non-null  object        
 2   state              74535 non-null  object        
 3   country            70662 non-null  object        
 4   shape              78400 non-null  object        
 5   duration_sec       80329 non-null  float64       
 6   duration_min_hour  80332 non-null  object        
 7   comments           80317 non-null  object        
 8   date_posted        80332 non-null  datetime64[ns]
 9   lat                80331 non-null  float64       
 10  long               80332 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 6.7+ MB


We also need to check for missing values.

In [31]:
#check for missing values in each column
df_ufo.isnull().sum()

date                    0
city                    0
state                5797
country              9670
shape                1932
duration_sec            3
duration_min_hour       0
comments               15
date_posted             0
lat                     1
long                    0
dtype: int64

In [32]:
#print sample of data where country is missing
df_ufo[df_ufo['country'].isnull()]

,date,city,state,country,shape,duration_sec,duration_min_hour,comments,date_posted,lat,long
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
18,1973-10-10 23:00:00,bermuda nas,NaN,NaN,light,20.0,20 sec.,saw fast moving blip on the radar scope thin w...,2002-01-11,32.364167,-64.678611
29,1979-10-10 22:00:00,saddle lake (canada),ab,NaN,triangle,270.0,4.5 or more min.,Lights far above&#44 that glance; then flee f...,2005-01-19,53.970571,-111.689885
35,1982-10-10 07:00:00,gisborne (new zealand),NaN,NaN,disk,120.0,2min,gisborne nz 1982 wainui beach to sponge bay,2002-01-11,-38.662334,178.017649
40,1986-10-10 20:00:00,holmes/pawling,ny,NaN,chevron,180.0,3 minutes,Football Field Sized Chevron with bright white...,2007-10-08,41.523427,-73.646795
...,...,...,...,...,...,...,...,...,...,...,...
80238,2009-09-09 14:15:00,broomfield?lafayette,co,NaN,rectangle,120.0,2 min,Large&#44 rectangular object seen flying in br...,2009-12-12,39.993596,-105.089706
80244,2009-09-09 20:17:00,lyman,me,NaN,light,600.0,10 mins,Two lights ran across the sky&#44 as bright as...,2009-12-12,43.505096,-70.637968
80319,2013-09-09 20:15:00,clifton,nj,NaN,other,3600.0,~1hr+,Luminous line seen in New Jersey sky.,2013-09-30,40.858433,-74.163755
80322,2013-09-09 21:00:00,aleksandrow (poland),NaN,NaN,light,15.0,15 seconds,Two points of light following one another in a...,2013-09-30,50.465843,22.891814


In [34]:
#impute 'us' to missing country value where state is not missing
df_ufo.loc[(df_ufo['country'].isnull()) & (~df_ufo['state'].isnull()), 'country'] = 'us'

In [50]:
#convert duration in sec to hours

# extract minute and hour column from duration_sec
df_ufo['duration_min'] = df_ufo['duration_sec'].apply(lambda x: x/60)

df_ufo['duration_hour'] = df_ufo['duration_sec'].apply(lambda x: x/(60*60))

In [51]:
#show top 5 records with the most duration
df_ufo.sort_values(by='duration_hour', ascending=False).head(5)

,date,city,state,country,shape,duration_sec,duration_min_hour,comments,date_posted,lat,long,year,duration_min,duration_hour
559,1983-10-01 17:00:00,birmingham (uk/england),NaN,Great Brittain,sphere,97836000.0,31 years,Firstly&#44 I was stunned and stared at the ob...,2013-04-12,52.466667,-1.916667,1983,1.630600e+06,27176.666667
53384,2010-06-03 23:30:00,ottawa (canada),on,Canada,other,82800000.0,23000hrs,((HOAX??)) I was out in a field near mil&#44 ...,2010-07-06,45.416667,-75.700000,2010,1.380000e+06,23000.000000
74660,1991-09-15 18:00:00,greenbrier,ar,United States,light,66276000.0,21 years,Orange or amber balls or orbs of light multipl...,2008-03-31,35.233889,-92.387500,1991,1.104600e+06,18410.000000
64390,2012-08-10 21:00:00,finley,wa,United States,light,52623200.0,2 months,There have been several flying objects in a pe...,2012-08-19,46.154167,-119.032778,2012,8.770533e+05,14617.555556
38261,1983-04-03 00:00:00,dont know,NaN,NaN,NaN,52623200.0,2 months,Hi&#44 I&#8217;m writing to you because I wan...,2004-07-25,41.730561,-78.682099,1983,8.770533e+05,14617.555556


It seems from the report comment that the duration of 3 years was indeed an error.

In [52]:
list(df_ufo[df_ufo['date'] == '1983-10-01 17:00:00']['comments'])

['Firstly&#44 I was stunned and stared at the object for what seemed minutes&#44 but probably was only seconds. My first inclination was to bec']

In [55]:
# count rows where duration hours is more than 24
df_ufo[df_ufo['duration_hour'] > 24].shape[0]

0

Reports with a duration of more than a day seems implausible. There are in total 179 of them. Let's remove them since our dataset is already large enough and it would only make our charts deteriorated.

In [56]:
# remove records with duration_hour more than 24
df_ufo = df_ufo[df_ufo['duration_hour'] <= 24]

## Data Inference using Plotly Express

Let's use Plotly Express to visualize the data and answer the business questions we layed out above.

### 1. Figure out which shapes of UFOs tend to be seen most often.

In [35]:
#create title and labels
title = 'Distribution of Reported Shapes'

labels = {'shape': 'Shape', 'count': 'Frequency'}

In [36]:
#calculate frwequency of shapes and keep top 10
dfg = df_ufo.groupby(['shape']).size().to_frame().sort_values([0], ascending = False).head(10).reset_index()

#we have to rename the columns because of multindexing
dfg.columns = ['shape', 'count']

#use plotly bar to create a bar chart and order by count
shapesbar = px.bar(dfg, x ='shape', y='count',
                    title = title, labels = labels
                    ).update_xaxes(categoryorder='total ascending')
shapesbar.show()

It seems that most of the reporters have seen UFOs in the form of lights in the sky. This is pretty vague and could be attributed to the fact that people tend to mistake anti-collison lights of airplanes for UFOs.

### 2. Take a look at data since the year 2000. How have annual sightings trended?

Next, let's see how annual sightings have tended over the years by countries

In [37]:
#since there are not much country in our data to make it, let's facet it by country
df_ufo['country'].value_counts()

us    71528
ca     3000
gb     1905
au      538
de      105
Name: country, dtype: int64

In [38]:
#rename country values to full name for legend labels
df_ufo = df_ufo.replace({'country' : { 'us' : 'United States', 'ca' : 'Canada', 'gb' : 'Great Brittain', 'au' : 'Australia', 'de' : 'Germany'}})

In [39]:
#create title and labels
title = 'Distribution of Reported Shapes by Country'

labels = {'country': 'Country', 'date': 'Date', 'count': 'Frequency'}

In [40]:
#add year column generated from date
df_ufo['year'] = df_ufo['date'].dt.year

# aggregate by year and country, plus add filter from 1990
dfg = df_ufo[df_ufo['year'] >= 1990].groupby(['country', 'year']).size().to_frame().reset_index()
dfg.columns = ['country', 'year', 'count']

#plot as linechart
countryline = px.line(dfg, x ='year', y='count', color='country', title=title, labels = labels)
countryline.show()

The conclusion we can make is that there large majority of the sightings are coming from the US (thousands per year). Excluding US from the plot, the second-bests are Canada and Australia with annual frequencies in the range of 100 to 300 per annum. The number of reports started to trend upwards after 1995 and been in decline over the recent years.

### 3. Which states in the US report the most UFO sightings?

In [57]:
# filter data to United States
df_ufo_us = df_ufo[df_ufo['country'] == 'United States']

In [58]:
#group count by states
dfg = df_ufo_us.groupby(['state']).size().to_frame().reset_index()
dfg.columns = ['state', 'count']

In [59]:
#create title and labels
title = 'Distribution of Reported Shapes by Country'

labels = {'state': 'Country', 'date': 'Date', 'count': 'Frequency'}

In [60]:
#check missing values in lat and long columns
df_ufo_us.isnull().sum()

date                    0
city                    0
state                   0
country                 0
shape                1736
duration_sec            0
duration_min_hour       0
comments               13
date_posted             0
lat                     1
long                    0
year                    0
duration_min            0
duration_hour           0
dtype: int64

In [61]:
#uppercase state abbreviations
df_ufo_us['state'] = df_ufo_us['state'].str.upper()

#calculate all numeric aggreagates for states
df_ufo_us_agg = df_ufo.sort_values('date').groupby('state').agg({'state': 'count', 'duration_hour': 'mean', 'shape':  ['first', 'last'], 'date' : ['first', 'last'], 'city': 'first'})

#group count by states and count
dfg = df_ufo_us.groupby(['state']).size().to_frame().reset_index()
dfg.columns = ['state', 'count']

/var/folders/rn/pb4lfv4d48x2_hps66yh6qk80000gn/T/ipykernel_35566/608144335.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
#calculate all numeric aggreagates for states
df_ufo_us_agg = df_ufo_us.sort_values('date').groupby('state').agg({'state': 'count',
                                                                    'duration_hour': 'mean',
                                                                    'shape':  ['first', 'last'],
                                                                    'year' : ['first', 'last'],
                                                                    'city': 'first'}).reset_index()

# treat multindex
df_ufo_us_agg.columns = df_ufo_us_agg.columns.get_level_values(0)
#rename columns
df_ufo_us_agg.columns = ['state', 'count', 'duration_hour_mean', 'shape_first', 'shape_last', 'year_first', 'year_last', 'city_first']

#capitalize texts
df_ufo_us_agg['city_first'] = df_ufo_us_agg['city_first'].str.capitalize()
df_ufo_us_agg['shape_first'] = df_ufo_us_agg['shape_first'].str.capitalize()
df_ufo_us_agg['shape_last'] = df_ufo_us_agg['shape_last'].str.capitalize()

In [91]:
df_ufo_us_agg.head()

,state,count,duration_hour_mean,shape_first,shape_last,year_first,year_last,city_first
0,AB,45,0.354920,Circle,Oval,1956,2014,Edmondton (canada)
1,AK,352,0.364006,Cigar,Flash,1936,2014,Eklutna
2,AL,688,0.274284,Fireball,Circle,1939,2014,Waterloo
3,AR,664,0.304850,Disk,Light,1950,2014,Arkansas (eastern; on us hwy 70 abt 75 m w/o m...
4,AZ,2680,0.329139,Flash,Light,1946,2014,Dome


In [92]:
#use plottly graph object to plot US state map

#customize hover over with state aggregates
for col in df_ufo_us_agg.columns:
    df_ufo_us_agg[col] = df_ufo_us_agg[col].astype(str)

df_ufo_us_agg['text'] = 'First shape: ' + df_ufo_us_agg['shape_first'] + '<br>' + 'Last shape: ' + df_ufo_us_agg['shape_last'] + '<br>' + \
    'First report: ' + df_ufo_us_agg['year_first'] + '<br>' + 'Last report: ' + df_ufo_us_agg['year_last'] + '<br>' + \
    'First city: ' + df_ufo_us_agg['city_first']

us_states = go.Figure(data=go.Choropleth(
    locations=dfg['state'], # Spatial coordinates
    z = df_ufo_us_agg['count'], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    autocolorscale=False,
    text=df_ufo_us_agg['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title = "Sightings",
))

us_states.update_layout(
    title_text = 'Distribution of Reported Sighting by State',
    geo_scope='usa', # limit map scope to USA
)

us_states.show()

4. Create a histogram to understand the distribution of the duration of UFO sightings.

In [ ]:
df_ufo.head()

In [ ]:
df_ufo['duration_hour'].describe()

In [ ]:
#binning of duration in hours
bins = np.arange(0, 25, 2)
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12-14', '14-16', '16-18', '18-20', '20-22', '22-24']
df_ufo['duration_hour_bin'] = pd.cut(df_ufo['duration_hour'], bins=bins, labels=labels).astype('string')

In [ ]:
df_ufo.info()

In [ ]:
#distribution of sightings by country
title = 'Distribution of Reported Sighting Durations by Country'

labels = {'country': 'Country', 'duration_hour_bin': 'Duration (hour)'}

In [ ]:
#count frequncy by duration hour bin
dfg = df_ufo.groupby(['duration_hour_bin']).size().to_frame().reset_index()
dfg.columns = ['duration_hour_bin', 'count']

The duration of sightings is dominated by reported hours less than 2 hours

In [ ]:
durationbar = px.bar(dfg, x ='duration_hour_bin', y='count',
                    title = title, labels = labels
                    )

durationbar.update_xaxes(categoryorder='array', categoryarray= ['0 to 2', '2 to 4', '4 to 6', '6 to 8', '8 to 10', '10 to 12', '12 to 14', '14 to 16', '16 to 18', '18 to 20', '20 to 22', '22 to 24'])
durationbar.show()

In [ ]:
# collect average durations per shape
df_ufo_shape_duration = df_ufo.groupby(['shape'])['duration_min'].mean().reset_index()

In [ ]:
#horizontal braplot for average durations per shape
title = 'Average Duration (min) of Reported Sightings by Shape'
labels = {'shape': 'Shape', 'duration_min': 'Duration (min)'}

In [ ]:
shapeduration = px.bar(df_ufo_shape_duration, x='shape', y='duration_min', title=title, labels=labels).update_xaxes(categoryorder='total ascending')

shapeduration.show()

## Dash

Setup layout

In [ ]:
# create the Dash app object
app = JupyterDash(__name__)

In [ ]:
# displaying the charts side-by-side
years= df_ufo.year.unique()
countries= df_ufo.country.unique()

# we need a style to use the CSS inline display mode and optionally setting a width
style2={'display': 'inline-block', 'width': '49%'}

# Put the grapsh within a div and assign the style
app = JupyterDash(__name__)
app.layout = html.Div(
[
    html.H1("UFO Sightings 🛸"),
    html.Div("Dashboard of UFO sightings between 1949 and 2014") , 
    dcc.RadioItems(id= 'country1', options=countries, value=countries[0]),
    dcc.RadioItems(id= 'year1', options=years, value=years[0]),
    html.Div(dcc.Graph(id='countrytrend'), style= style2), 
    html.Div(dcc.Graph(id='shapes'), style= style2),
    html.Div(dcc.Graph(id='usmap'), style= style2)
 ]
)

# Callback with a single input parameter, returning the same figure twice
@app.callback(
    Output('countrytrend', 'figure'),
    Output('shapes', 'figure'),
    Output('usmap', 'figure'),
    Input('country1', 'value'),
    Input('year1', 'value')
)
def update1( _input1, _input2 ):
    # first figure 
    df1= df_ufo.query('year==@_input1 and country == @_input2')
    fig1= px.scatter(df1, x='total_bill', y="tip", color='sex')
    fig1.update_layout(title= "Showing values for Lunch on " +  _input1)

    # second figure  
    df2= df_ufo.query('year==@_input1 and country == @_input2')
    fig2= px.scatter(df2, x='total_bill', y="tip", color='sex')
    fig2.update_layout(title= "Showing values for Dinner on " +  _input1 + _input2)

    # third figure  
    df3= df_ufo.query(''year==@_input1 and country == @_input2')
    fig3= px.scatter(df3, x='total_bill', y="tip", color='sex')
    fig3.update_layout(title= "Showing values for Dinner on " +  _input1 + _input2)

    return fig1, fig2, fig3

app.run_server(mode='inline')